# Huấn luyện mô hình chủ đề (LDA) từ `merged.json`

Notebook này giúp bạn:
- Nạp dữ liệu từ `merged.json`, tiền xử lý văn bản
- Train LDA với `scikit-learn`
- Lưu model (`lda_model.joblib`), vectorizer (`vectorizer_bow.joblib`)
- Xuất chủ đề (`topics.json`, `topics.txt`) và gán chủ đề cho từng bài (`articles_with_topics.csv`)
- Tùy chọn sinh tóm tắt extractive cho mỗi chủ đề

Chạy lần lượt các cell bên dưới. Có thể chỉnh các tham số trong cell Cấu hình.


In [ ]:
# Cài đặt thư viện nếu thiếu (chỉ chạy nếu cần)
# !pip install pandas scikit-learn beautifulsoup4 joblib

import json
import os
import re
from typing import List

import joblib
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cấu hình
DATA_PATH = "merged.json"
OUTPUT_DIR = "models"
NUM_TOPICS = 12
MAX_FEATURES = 30000
MIN_DF = 3
MAX_DF = 0.9
RANDOM_STATE = 42
TOP_WORDS = 15
SUMMARIZE_TOPICS = True
TOPIC_SUMMARY_SENTENCES = 5
TOPIC_SUMMARY_MAX_ARTICLES = 200

os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
def strip_html(text: str) -> str:
    if not isinstance(text, str):
        return ""
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text(separator=" ")


def basic_clean(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = strip_html(text)
    text = re.sub(r"https?://\S+", " ", text)
    text = re.sub(r"www\.\S+", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def load_articles(path: str) -> pd.DataFrame:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    rows = []
    for item in data:
        if not isinstance(item, dict):
            continue
        title = item.get("title") or item.get("headline") or item.get("name")
        desc = item.get("description") or item.get("summary")
        content = item.get("content") or item.get("body") or item.get("text")
        url = item.get("url") or item.get("link")
        category = item.get("category") or item.get("section") or item.get("cat")
        published = (
            item.get("publishedAt")
            or item.get("pubDate")
            or item.get("date")
            or item.get("created_at")
        )
        full_text = " ".join([str(title or ""), str(desc or ""), str(content or "")]).strip()
        rows.append(
            {
                "title": title,
                "description": desc,
                "content": content,
                "url": url,
                "category": category,
                "published": published,
                "text": basic_clean(full_text),
            }
        )
    df = pd.DataFrame(rows)
    df = df[df["text"].astype(str).str.len() > 0].reset_index(drop=True)
    df.head(3)
    return df


df = load_articles(DATA_PATH)
print(df.shape)


(7050, 7)


In [ ]:
# Vector hóa và huấn luyện LDA
vectorizer = CountVectorizer(
    max_features=MAX_FEATURES,
    min_df=MIN_DF,
    max_df=MAX_DF,
)
X_bow = vectorizer.fit_transform(df["text"])  # sparse matrix

lda = LatentDirichletAllocation(
    n_components=NUM_TOPICS,
    learning_method="batch",
    random_state=RANDOM_STATE,
)
W = lda.fit_transform(X_bow)
H = lda.components_

terms = np.array(vectorizer.get_feature_names_out())

def top_words_for_topic(topic_idx: int, n: int = TOP_WORDS) -> List[str]:
    idx = np.argsort(H[topic_idx])[::-1][:n]
    return terms[idx].tolist()

for k in range(NUM_TOPICS):
    print(f"Chủ đề {k}: ", ", ".join(top_words_for_topic(k)))


Chủ đề 0:  bệnh, thể, các, cơ, người, không, chất, khi, hoặc, ăn, là, sĩ, cho, thường, bác
Chủ đề 1:  mỹ, ông, cho, không, đã, một, các, là, với, này, khi, thống, được, trump, ngày
Chủ đề 2:  không, là, bị, án, đề, được, cho, một, câu, các, với, học, để, này, đã
Chủ đề 3:  công, các, người, ai, cho, một, thể, dụng, với, được, động, hình, trên, năng, ra
Chủ đề 4:  trận, đội, bóng, thủ, đấu, là, với, khi, hai, anh, giải, thắng, sân, một, kết
Chủ đề 5:  tôi, không, người, một, là, con, khi, nhưng, để, làm, cho, nhà, anh, lại, vì
Chủ đề 6:  với, là, cho, giá, các, hơn, nước, không, độ, một, được, đồng, từ, bản, khi
Chủ đề 7:  học, sinh, thi, trường, điểm, đại, năm, các, là, viên, với, công, thí, giáo, được
Chủ đề 8:  xe, người, bị, đường, một, khi, trên, an, máy, đi, không, vào, sau, sát, đã
Chủ đề 9:  công, các, doanh, chính, cho, với, hàng, đồng, định, được, là, đầu, số, kinh, năm
Chủ đề 10:  khách, là, du, các, được, ảnh, với, đến, từ, một, những, năm, diễn, người, nhiều
Chủ đề 11:  tỉn

In [ ]:
# Lưu model và vectorizer
joblib.dump(vectorizer, os.path.join(OUTPUT_DIR, "vectorizer_bow.joblib"))
joblib.dump(lda, os.path.join(OUTPUT_DIR, "lda_model.joblib"))

# Lưu danh sách từ khóa mỗi chủ đề
topics = {}
for i in range(NUM_TOPICS):
    topics[str(i)] = top_words_for_topic(i, TOP_WORDS)
with open(os.path.join(OUTPUT_DIR, "topics.json"), "w", encoding="utf-8") as f:
    json.dump(topics, f, ensure_ascii=False, indent=2)
with open(os.path.join(OUTPUT_DIR, "topics.txt"), "w", encoding="utf-8") as f:
    for i in range(NUM_TOPICS):
        f.write(f"Topic {i}: ")
        f.write(", ".join(top_words_for_topic(i, TOP_WORDS)))
        f.write("\n")

print("Đã lưu model, vectorizer và topics vào:", os.path.abspath(OUTPUT_DIR))


Đã lưu model, vectorizer và topics vào: d:\Project\cc\models


In [ ]:
# Gán chủ đề trội cho từng bài và lưu CSV

dominant_topic = W.argmax(axis=1)
max_topic_score = W.max(axis=1)

df_topics = df.copy()
df_topics["dominant_topic"] = dominant_topic
[df_topics.__setitem__("dominant_topic_score", max_topic_score)]

cols = ["title", "url", "category", "published", "dominant_topic", "dominant_topic_score"]
for c in cols:
    if c not in df_topics.columns:
        df_topics[c] = None

out_csv = os.path.join(OUTPUT_DIR, "articles_with_topics.csv")
df_topics[cols].to_csv(out_csv, index=False)
print("Đã lưu:", out_csv)

df_topics.head(3)


Đã lưu: models\articles_with_topics.csv


,title,description,content,url,category,published,text,dominant_topic,dominant_topic_score
0,Cao thủ duy nhất trong Kim Dung chết do võ côn...,Đây là một nhân vật đầy quyền lực và mưu mô tr...,Ông là giáo chủ đầy tham vọng của Nhật Nguyệt ...,https://vnexpress.net/crossword-giai-o-chu-o-c...,cuoi,None,Cao thủ duy nhất trong Kim Dung chết do võ côn...,2,0.606936
1,Ảnh phòng tắm có điểm sai duy nhất nào?,Chỉ với 30 giây bạn có nhận ra điểm thiếu sót ...,Trong bức ảnh miêu tả căn phòng tắm với đầy đủ...,https://vnexpress.net/cau-do-iq-thu-tai-tinh-m...,cuoi,None,Ảnh phòng tắm có điểm sai duy nhất nào? Chỉ vớ...,2,0.492050
2,Triều đại cuối cùng đóng đô tại hai kinh đô kh...,"Triều đại này tồn tại khoảng 24 năm, từng đóng...","Trước khi sụp đổ, triều đại này còn dự định đó...",https://vnexpress.net/crossword-giai-o-chu-o-c...,cuoi,None,Triều đại cuối cùng đóng đô tại hai kinh đô kh...,10,0.429586


In [ ]:
# Tóm tắt extractive cho mỗi chủ đề (tùy chọn)

def split_sentences(text: str) -> List[str]:
    sentences = re.split(r"(?<=[\.\?\!])\s+", text)
    return [s.strip() for s in sentences if s and len(s.strip()) > 3]


def summarize_text(text: str, max_sentences: int = 5) -> str:
    sentences = split_sentences(text)
    if len(sentences) <= max_sentences:
        return " ".join(sentences)
    vec = TfidfVectorizer(max_features=MAX_FEATURES)
    tfidf = vec.fit_transform(sentences)
    sim = cosine_similarity(tfidf)
    n = sim.shape[0]
    scores = np.ones(n) / n
    damping = 0.85
    for _ in range(30):
        denom = np.maximum(sim.sum(axis=1), 1e-9)
        new_scores = (1 - damping) / n + damping * sim.dot(scores) / denom
        if np.allclose(new_scores, scores, atol=1e-6):
            break
        scores = new_scores
    top_idx = np.argsort(scores)[::-1][:max_sentences]
    top_idx = sorted(top_idx)
    return " ".join(sentences[i] for i in top_idx)


if SUMMARIZE_TOPICS:
    lines = []
    for k in sorted(df_topics["dominant_topic"].unique()):
        subset = df_topics[df_topics["dominant_topic"] == k]
        texts = subset["text"].head(TOPIC_SUMMARY_MAX_ARTICLES).tolist()
        combined = ". ".join(texts)
        summary = summarize_text(combined, max_sentences=TOPIC_SUMMARY_SENTENCES)
        lines.append(f"==== TOPIC {k} ====")
        lines.append(summary)
        lines.append("")

    with open(os.path.join(OUTPUT_DIR, "topic_summaries.txt"), "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    print("Đã lưu tóm tắt chủ đề vào:", os.path.join(OUTPUT_DIR, "topic_summaries.txt"))


Đã lưu tóm tắt chủ đề vào: models\topic_summaries.txt


In [ ]:
# Tóm tắt 1 bài theo chỉ số (index)
# Đổi giá trị idx để chọn bài cần tóm tắt
idx = 0  # ví dụ: bài đầu tiên

if 0 <= idx < len(df):
    print("Tiêu đề:", df.loc[idx, "title"])
    print("URL:", df.loc[idx, "url"])
    summary_single = summarize_text(df.loc[idx, "text"], max_sentences=3)
    print("\nTóm tắt:")
    print(summary_single)
else:
    print("idx ngoài phạm vi dữ liệu")


Tiêu đề: Cao thủ duy nhất trong Kim Dung chết do võ công mình tạo ra?
URL: https://vnexpress.net/crossword-giai-o-chu-o-chu-cao-thu-duy-nhat-trong-kim-dung-chet-do-vo-cong-minh-tao-ra-4905407.html

Tóm tắt:
Tuy nhiên, chính tuyệt kỹ này lại trở thành nguyên nhân dẫn đến cái chết bất ngờ của ông. Gợi ý:Ông là cha của Nhậm Doanh Doanh, tên đầy đủ có 11 chữ cái, bắt đầu là chữ N và kết thúc là chữ H. >> Xem video hướng dẫn cách giải ô chữ tại đây >> Xem đáp án Hi


In [ ]:
# Tóm tắt nhanh toàn bộ 1 chủ đề theo topic_id
# Đổi topic_id để chọn chủ đề cần xem tóm tắt

def summarize_topic(topic_id: int, max_sentences: int = 5, max_articles: int = 200) -> str:
    subset = df_topics[df_topics["dominant_topic"] == topic_id]
    if subset.empty:
        return "(Không có bài phù hợp)"
    texts = subset["text"].head(max_articles).tolist()
    combined = ". ".join(texts)
    return summarize_text(combined, max_sentences=max_sentences)

# ví dụ: tóm tắt chủ đề 0
topic_id = 0
print(f"==== TÓM TẮT CHỦ ĐỀ {topic_id} ====")
print(summarize_topic(topic_id, max_sentences=5))


==== TÓM TẮT CHỦ ĐỀ 0 ====
Những thứ không xả xuống bồn cầu? Tuy nhiên, nếu hộp không chịu được nhiệt cao (thường là hộp nhựa thông thường, không chuyên dụng cho nhiệt), việc tích nhiệt và áp suất lâu bên trong có thể gây ra hiện tượng biến dạng, bung nắp, hoặc xuất hiện các vết xước nhỏ ở đáy – nơi dễ phát sinh vi khuẩn khi tái sử dụng. Kíp cấp cứu đã đến bến tàu, tiếp cận người bệnh vào khoảng 9h50 trong tình trạng đang được cấp cứu hồi sinh tim phổi cơ bản, vùng hầu họng nhiều thức ăn và dịch tiêu hóa, mạch cánh đập rời rạc. Anna Strasma, bác sĩ tại Đại học Duke, cho rằng CKDu có thể là tập hợp các bệnh với nhiều nguyên nhân khác nhau. Ảnh:Dreamstime Các nhà nghiên cứu thừa nhận một số hạn chế trong nghiên cứu, như khả năng không đại diện cho các nhóm dân số khác và các yếu tố nguy cơ khác như hút thuốc có thể ảnh hưởng đến kết quả.
